In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Contour 이미지가 zipfile로 존재할 경우 압축 해제하고 진행

import zipfile

with zipfile.ZipFile('/content/drive/MyDrive/Likenot_ML/Bra_Contours.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Likenot_ML')

# 1. 원본 이미지 전처리(Canny & Resize)


In [ ]:
# 1. 원본 이미지 zip 압축 해제 (압축 해제되어 있을 경우 이 과정은 건너뛴다)

import zipfile

# !! 압축 해제한 원본 이미지 저장 경로 설정 !!
src_dir = '/content/drive/MyDrive/Likenot_ML/bratabase_model' 

with zipfile.ZipFile('/content/drive/MyDrive/Likenot_ML/bratabase_model.zip', 'r') as zip_ref:
    zip_ref.extractall(src_dir)

In [ ]:
# 2. Canny edge 이미지에 Black Border(검은색 여백)을 추가하여 정사각형 비율을 맞춘 후에
# (256, 256)으로 resize한 이미지를 반환하는 함수

import cv2

def resize_image(img, resize_shape): # resize_shape: 세로, 가로 길이가 같은 정사각형의 튜플
    y = img.shape[0] # 이미지의 세로 길이
    x = img.shape[1] # 이미지의 가로 길이

    if y < x: # 가로가 세로보다 길 경우 위, 아래에 Black Border을 추가한다
        img_border = cv2.copyMakeBorder(img, int((x-y)/2), int((x-y)/2), 0, 0, cv2.BORDER_CONSTANT, 0)
    else: # 세로가 가로보다 길 경우 양 옆에 Black Border을 추가한다
        img_border = cv2.copyMakeBorder(img, 0, 0, int((y-x)/2), int((y-x)/2), cv2.BORDER_CONSTANT, 0)

    # 정사각형 비율의 이미지 img_border을 shape에 맞춰 resize한다   
    img_resized = cv2.resize(img_border, resize_shape, interpolation=cv2.INTER_AREA)

    return img_resized

In [ ]:
# 3. 전처리 이미지를 저장할 폴더 생성 (이미 존재하는 폴더일 경우 새로 생성되지 않는다)
!mkdir '/content/drive/MyDrive/Likenot_ML/Bra_canny'

In [ ]:
# 4. 원본 이미지에서 Canny edge 검출 후 resize 진행

import os
import cv2

# !! 원본 이미지 폴더 경로 설정 !!
src_dir = '/content/drive/MyDrive/Likenot_ML/Bra_Image_edit'
src_image_list = os.listdir(src_dir) # 원본 이미지의 파일 리스트

# !! 전처리 이미지 저장할 폴더 경로 설정 !!
dst_dir = '/content/drive/MyDrive/Likenot_ML/Bra_canny'

# 원본 이미지 전처리 과정
for file in src_image_list:
    src_path = src_dir + '/' + file # 원본 이미지 경로
    img = cv2.imread(src_path, 0)

    # 원본 이미지에서 Canny edge 검출
    canny= cv2.Canny(img, 30, 70)
    # Canny edge 이미지 resize 진행
    resized_img = resize_image(canny, (256, 256))

    dst_path = dst_dir + '/' + file # 전처리 이미지 저장 경로
    cv2.imwrite(dst_path, resized_img)

# 2. 전처리 이미지에서 Contour(윤곽선) 추출

In [ ]:
# 1. ndarray인 Contour을 이미지로 반환하는 함수

import numpy as np

def contour2img(contours):
    # 검은 배경 이미지 형성
    contour_img = np.zeros(shape=(256, 256))

    for contour in contours:
        line = contour.reshape(len(contour), 2)
        for pixel in line:
            # Contour에 해당하는 pixel을 흰 색으로 저장
            contour_img[pixel[1], pixel[0]] = 255 

    return contour_img

In [ ]:
# 2. Contour 이미지를 저장할 폴더 생성 (이미 존재하는 폴더일 경우 생성되지 않는다)
!mkdir /content/drive/MyDrive/Likenot_ML/Bra_Contours

In [ ]:
# 3. Canny edge 이미지에서 외곽선을 구성하는 contour을 추출하여 이미지로 저장한다

import cv2
import os
import numpy as np

# !! 전처리 이미지 폴더 경로 설정 !!
canny_dir = '/content/drive/MyDrive/Likenot_ML/Bra_canny'

# !! Contour 이미지 저장할 폴더 경로 설정 !!
dst_dir = '/content/drive/MyDrive/Likenot_ML/Bra_Contours'

# 전처리 이미지 파일 리스트
img_list = os.listdir(canny_dir)

for image in img_list:
    canny = canny_dir + '/' + image
    img = cv2.imread(canny)
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # contours 추출
    contours, hierarchy = cv2.findContours(imgray, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    # 이미지의 형태를 나타내는 외곽선만 추출해야 하므로 
    # contour을 구성하는 점의 개수가 많은, 즉 길이가 긴 순서대로 contours를 정렬한다
    contour_order = np.argsort([x.shape[0] for x in contours])[::-1]
    # 가장 길이가 긴 contour 3개를 외곽선으로 정의한다
    outside = [contours[x] for x in contour_order[:3]]

    # 추출한 contour 3개를 이미지로 반환한다
    contour_img = contour2img(outside)

    dst = dst_dir + '/' + image
    cv2.imwrite(dst, contour_img)

# 3. Contour 이미지 라벨링

In [ ]:
# 1. !! csv 파일, Contour 이미지 폴더 경로 설정 !!
csv_dir = '/content/drive/MyDrive/Likenot_ML/Image_Label.csv'
contour_dir = '/content/drive/MyDrive/Likenot_ML/Bra_Contours'

In [ ]:
# 2. 이미지 라벨 불러오기

# jamo 패키지 설치
!pip install jamo

from jamo import h2j
import pandas as pd
import os

# Image_Label.csv를 읽어들인다
label = pd.read_csv(csv_dir)

# 이미지 파일의 str과 image_label['ID']의 str의 한글 자모 조합 형태를 통일시킨다
label['ID'] = [h2j(id) for id in label['ID']]

# 측정하지 않은 고어 너비(결측값)을 정수 -1로 변환
label = label.fillna(-1)
label['goreWidth'] = label['goreWidth'].astype(int)

# label에서 Contour 이미지 폴더에 파일이 존재하는 행만 골라낸다
image_list = os.listdir(contour_dir)
image_label = pd.DataFrame(columns=label.columns)

for file in image_list:
    image_id = file.split('.')[0] # 확장자 제거
    
    # 이미지 폴더에 파일이 존재하는 image_id만으로 image_label dataframe을 재구성한다
    if image_id in list(label['ID']):
        image_label = image_label.append(label[label['ID'] == image_id], ignore_index=True)

image_label.head()

,ID,Cup,goreHeight,goreWidth,isBraret
0,프리부_8,100,2,-1,1
1,휠라_8,100,2,1,0
2,휠라_1,100,0,0,0
3,프리부_1,75,0,-1,1
4,프리부_3,75,0,-1,1


In [ ]:
# 3. 이미지의 label을 기준으로 train, test set을 반환하는 함수

import numpy as np
from sklearn.model_selection import StratifiedKFold

def train_test_set(column): # column: label 이름
    # test_size=0.2 이므로 n_splits=5
    skf = StratifiedKFold(n_splits=5)
    
    # skf.split의 마지막 train, test index를 기준으로 train, test set을 나눈다
    for train, test in skf.split(list(image_label['ID']), list(image_label[column])):
        train_index = train
        test_index = test
    
    X_train = list(image_label['ID'].iloc[train_index])
    X_test = list(image_label['ID'].iloc[test_index])
    y_train = list(image_label[column].iloc[train_index])
    y_test = list(image_label[column].iloc[test_index])

    return X_train, X_test, y_train, y_test

In [ ]:
# 4. 라벨링한 Contour 이미지를 저장할 폴더 생성 (이미 존재할 경우 생성되지 않는다)
!mkdir /content/drive/MyDrive/Likenot_ML/Bra_Contours_Labeled

In [ ]:
# 5. !! 라벨링한 이미지를 저장할 폴더 경로 설정 !!
dst_dir = '/content/drive/MyDrive/Likenot_ML/Bra_Contours_Labeled'

In [ ]:
# 6. 'isBraret' label을 기준으로 train, test set을 나눈 후 라벨링
# 1 -> true / 0 -> false
import zipfile

# train test split
X_train, X_test, braret_train, braret_test = train_test_set('isBraret')

# 이미지 저장 압축 파일 설정
dst_zip = dst_dir + '/isBraret.zip'

for file in image_list:
    srcname = contour_dir + '/' + file # 전처리 이미지 경로
    image_id = file.split('.')[0] # 확장자 제거한 파일명
    # train set 형성
    if image_id in X_train:
        # train set 내 이미지 라벨링
        if braret_train[X_train.index(image_id)] == 1:
            arcname = '/train/true/' + file
        else:
            arcname = '/train/false/' + file
        
        with zipfile.ZipFile('%s'%(dst_zip), 'a') as bra:
            bra.write(srcname, arcname)

    # test set 형성
    elif image_id in X_test:
        # test set 내 이미지 라벨링
        if braret_test[X_test.index(image_id)] == 1:
            arcname = '/test/true/' + file
        else:
            arcname = '/test/false/' + file
        
        with zipfile.ZipFile('%s'%(dst_zip), 'a') as bra:
            bra.write(srcname, arcname)

In [ ]:
# 7. 'Cup' label을 기준으로 train, test set을 나눈 후 라벨링
# 100 -> 풀컵 / 75 -> 3/4컵 / 50 -> 1/2컵
import zipfile

# train test split
X_train, X_test, cup_train, cup_test = train_test_set('Cup')

# 이미지 저장 압축 파일 설정
dst_zip = dst_dir + '/Cup.zip'

for file in image_list:
    srcname = contour_dir + '/' + file # Contour 이미지 경로
    image_id = file.split('.')[0] # 확장자 제거한 파일명
    # train set 형성
    if image_id in X_train:
        # train set 내 이미지 라벨링
        if cup_train[X_train.index(image_id)] == 100:
            arcname = '/train/100/' + file
        elif cup_train[X_train.index(image_id)] == 75:
            arcname = '/train/75/' + file
        else:
            arcname = '/train/50/' + file
        
        with zipfile.ZipFile('%s'%(dst_zip), 'a') as bra:
            bra.write(srcname, arcname)

    # test set 형성
    elif image_id in X_test:
        # test set 내 이미지 라벨링
        if cup_test[X_test.index(image_id)] == 100:
            arcname = '/test/100/' + file
        elif cup_test[X_test.index(image_id)] == 75:
            arcname = '/test/75/' + file
        else:
            arcname = '/test/50/' + file
        
        with zipfile.ZipFile('%s'%(dst_zip), 'a') as bra:
            bra.write(srcname, arcname)

In [ ]:
# 8. 'goreHeight' label을 기준으로 train, test set을 나눈 후 라벨링
# 2 -> high / 1 -> middle / 0 -> low
import zipfile

# train test split
X_train, X_test, goreH_train, goreH_test = train_test_set('goreHeight')

# 이미지 저장 압축 파일 설정
dst_zip = dst_dir + '/goreHeight.zip' 

for file in image_list:
    srcname = contour_dir + '/' + file # Contour 이미지 경로
    image_id = file.split('.')[0] # 확장자 제거한 파일명
    # train set 형성
    if image_id in X_train:
        # train set 내 이미지 라벨링
        if goreH_train[X_train.index(image_id)] == 2:
            arcname = '/train/high/' + file
        elif goreH_train[X_train.index(image_id)] == 1:
            arcname = '/train/middle/' + file
        else:
            arcname = '/train/low/' + file
        
        with zipfile.ZipFile('%s'%(dst_zip), 'a') as bra:
            bra.write(srcname, arcname)

    # test set 형성
    elif image_id in X_test:
        # test set 내 이미지 라벨링
        if goreH_test[X_test.index(image_id)] == 2:
            arcname = '/test/high/' + file
        elif goreH_test[X_test.index(image_id)] == 1:
            arcname = '/test/middle/' + file
        else:
            arcname = '/test/low/' + file
        
        with zipfile.ZipFile('%s'%(dst_zip), 'a') as bra:
            bra.write(srcname, arcname)

In [ ]:
# 9. 'goreWidth' label을 기준으로 train, test set을 나눈 후 라벨링
# 2 -> wide / 1 -> middle / 0 -> narrow / -1 -> 측정X
import zipfile

# train test split
X_train, X_test, goreW_train, goreW_test = train_test_set('goreWidth')

# 이미지 저장 압축 파일 설정
dst_zip = dst_dir + '/goreWidth.zip'

for file in image_list:
    srcname = contour_dir + '/' + file # Contour 이미지 경로
    image_id = file.split('.')[0] # 확장자 제거한 파일명
    # train set 형성
    if image_id in X_train:
        # train set 내 이미지 라벨링
        if goreW_train[X_train.index(image_id)] == 2:
            arcname = '/train/wide/' + file
        elif goreW_train[X_train.index(image_id)] == 1:
            arcname = '/train/middle/' + file
        elif goreW_train[X_train.index(image_id)] == 0:
            arcname = '/train/narrow/' + file
        else:
            arcname = '/train/none/' + file
        
        with zipfile.ZipFile('%s'%(dst_zip), 'a') as bra:
            bra.write(srcname, arcname)

    # test set 형성
    elif image_id in X_test:
        # test set 내 이미지 라벨링
        if goreW_test[X_test.index(image_id)] == 2:
            arcname = '/test/wide/' + file
        elif goreW_test[X_test.index(image_id)] == 1:
            arcname = '/test/middle/' + file
        elif goreW_test[X_test.index(image_id)] == 0:
            arcname = '/test/narrow/' + file
        else:
            arcname = '/test/none/' + file
        
        with zipfile.ZipFile('%s'%(dst_zip), 'a') as bra:
            bra.write(srcname, arcname)